In [26]:
__author__ = 'SherlockLiao'

import torch
import time
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
#batch_size = 600
learning_rate = 1e-2
num_epoches = 15

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 定义简单的前馈神经网络
"""
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
"""
model = nn.Sequential(
    torch.nn.Linear(784, 300),
    torch.nn.ReLU(),
    torch.nn.Linear(300, 10)

)

# model = Neuralnetwork(28 * 28, 300, 100, 10)
usegpu = torch.cuda.is_available()
#usegpu = False
if usegpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if usegpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            #img = Variable(img)
            #label = Variable(label)
            img = Variable(img, requires_grad=True)
            label = Variable(label, requires_grad=True)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc * 100.0 / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc * 100.0 / (len(
            train_dataset))))
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if usegpu:
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc * 100.0 / (len(test_dataset))))
    print('Time:{:.1f} s'.format(time.time() - since))
    # print()

# 保存模型
torch.save(model.state_dict(), './neural_network.pth')


epoch 1
**********


D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:77: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:80: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/15] Loss: 1.915676, Acc: 61.000000
[1/15] Loss: 1.501811, Acc: 70.000000
[1/15] Loss: 1.235979, Acc: 75.000000
[1/15] Loss: 1.069895, Acc: 77.000000
[1/15] Loss: 0.952911, Acc: 79.000000
[1/15] Loss: 0.868519, Acc: 81.000000
Finish 1 epoch, Loss: 0.851153, Acc: 81.000000


D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:101: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:102: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:108: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
D:\Anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:111: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 0.401144, Acc: 89.000000
Time:9.3 s
epoch 2
**********
[2/15] Loss: 0.410774, Acc: 88.000000
[2/15] Loss: 0.401337, Acc: 89.000000
[2/15] Loss: 0.394904, Acc: 89.000000
[2/15] Loss: 0.385124, Acc: 89.000000
[2/15] Loss: 0.378577, Acc: 89.000000
[2/15] Loss: 0.373242, Acc: 89.000000
Finish 2 epoch, Loss: 0.371321, Acc: 89.000000
Test Loss: 0.319316, Acc: 90.000000
Time:8.6 s
epoch 3
**********
[3/15] Loss: 0.340581, Acc: 90.000000
[3/15] Loss: 0.334892, Acc: 90.000000
[3/15] Loss: 0.326904, Acc: 90.000000
[3/15] Loss: 0.324488, Acc: 90.000000
[3/15] Loss: 0.320890, Acc: 90.000000
[3/15] Loss: 0.318455, Acc: 90.000000
Finish 3 epoch, Loss: 0.316985, Acc: 91.000000
Test Loss: 0.284428, Acc: 92.000000
Time:8.3 s
epoch 4
**********
[4/15] Loss: 0.289783, Acc: 91.000000
[4/15] Loss: 0.291071, Acc: 91.000000
[4/15] Loss: 0.295104, Acc: 91.000000
[4/15] Loss: 0.292102, Acc: 91.000000
[4/15] Loss: 0.288148, Acc: 91.000000
[4/15] Loss: 0.286724, Acc: 91.000000
Finish 4 epoch, Loss: 0.

KeyboardInterrupt: 

In [36]:
a = 100
b = 0
print(int(a))
print(float(a))
print(bool(a))

100
100.0
True


In [44]:
a = 100
b = 0
print(a and b)
print(a & b)
print(a or b)
print(a | b)

0
0
100
100


In [45]:
a = 1
not a

False

In [4]:
b

3.0

In [5]:
a

1

In [8]:
import torch
autograd_tensor = torch.randn((2, 3, 4), requires_grad=True)
autograd_tensor

tensor([[[-0.0525, -0.2310,  0.1846,  0.6891],
         [ 1.7906,  0.0752,  0.0124, -1.9266],
         [-0.5039,  0.4892,  0.2209, -0.2833]],

        [[ 1.0047, -0.7062,  1.7337, -0.1104],
         [-0.0676, -0.2381,  0.8615, -0.7643],
         [-0.3251, -0.2494,  1.6583, -0.2700]]], requires_grad=True)

In [10]:
torch.randn((2, 3, 4))

tensor([[[ 0.3648, -1.5205, -0.2521, -0.9341],
         [-0.2465,  0.3172, -1.1822, -1.2198],
         [ 0.5593,  1.4876,  0.8035, -1.2193]],

        [[-1.1396,  0.4503,  0.3615, -2.6644],
         [ 0.8629,  1.6920, -0.3163,  1.1178],
         [-0.1792,  0.5120, -0.6313,  0.0295]]])

In [44]:
import torch
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
learning_rate = 1e-2
num_epoches = 50


train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)

In [45]:
img

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [48]:
Variable(img, requires_grad=True)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [51]:
Variable(img, requires_grad=True).cuda()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]],
       device='cuda:0', grad_fn=<CopyBackwards>)

In [49]:
torch.tensor(img, requires_grad=True).cuda()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]],
       device='cuda:0', grad_fn=<CopyBackwards>)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]],
       device='cuda:0', grad_fn=<CopyBackwards>)

In [20]:
torch.tensor([0., 0., 0.], requires_grad=True)

tensor([0., 0., 0.], requires_grad=True)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [38]:
torch.device('cuda:0')

device(type='cuda', index=0)

In [8]:
a = 100.0
b = 0.1
print('{:.6f}'.format(a-b))

99.900000


In [17]:
a = 100
b = 12
a/b

8.333333333333334

In [2]:
import torch
import time
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 32
#batch_size = 600
learning_rate = 1e-2
num_epoches = 15

train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [7]:
train_loader.dataset.train_data.shape

torch.Size([60000, 28, 28])